# 02d – 5. Experiment: Modell-Vergleich: Whisper-Flamingo (Audio-Visuell)

## Motivation

`02c` zeigte, dass reines Audio-ASR (Whisper Large-v3) in der überlappenden
Mehrsprecher-Umgebung von MCoRec stark versagt (WER ~1.03). Als nächster Schritt
wird **Whisper-Flamingo** getestet – eine Erweiterung von Whisper, die zusätzlich
visuelle Features (Lip-Crops) als Input verarbeitet.

**Architektur:** Whisper-Flamingo kombiniert den Whisper-Encoder/-Decoder mit
einem visuellen Feature-Extraktor. Der Checkpoint
`whisper-flamingo_en_large_vc2_noisy.pt` wurde auf englischen AV-Daten trainiert.

## Experiment-Design

Analoges Grid wie in `02_` für BL4: `beam_size ∈ {4, 8, 12}` × `max_length ∈ {10, 15, 18, 20}` s.
So ist ein direkter Hyperparameter-Vergleich mit BL4 und Whisper möglich.

## Ergebnis (Vorschau)

| Modell | WER | Joint Error |
|--------|-----|-------------|
| Whisper Large-v3 (audio-only) | ~1.03 | ~0.59 |
| Whisper-Flamingo (beste Konfig.) | ~0.92 | ~0.54 |
| BL4 AV-HuBERT (beam=12, len=20) | ~0.495 | ~0.324 |

Whisper-Flamingo verbessert sich ~10 % relativ gegenüber Whisper,
bleibt aber **weit hinter BL4**. AV-HuBERT mit In-Domain-Fine-Tuning bleibt
klar überlegen.

**Hinweis zum Bugfix:** Dieser Lauf wurde **vor dem Bugfix** in `segmentation.py` durchgeführt (`min_duration_off` las fälschlicherweise den Wert von `min_duration_on`). Das ist **gewollt**: Der Bugfix wurde erst nach Abschluss der LLM- und Hyperparameter-Experimente entdeckt. Da der Bugfix allein die WER zunächst verschlechterte, wurde erst in `02j_`/`02k_` die Kombination aus Bugfix + `min_duration`-Optimierung erarbeitet, die schließlich das beste Ergebnis lieferte.


## 1 – Setup: Arbeitsverzeichnis & Imports

In [1]:
import os, sys
import pandas as pd

# Arbeitsverzeichnis auf Repo-Root setzen (Voraussetzung für alle relativen Pfade)
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

# Repo-Root in sys.path, damit projektinterne Module importierbar sind
if project_baseline_path not in sys.path:
    sys.path.append(project_baseline_path)

from script.pg_utils_experiments import run_inference_for_experiment, run_eval_and_log, append_eval_results_for_experiments

/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:20: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "bool"):
/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:22: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:24: FutureWarning: In the future `np.long` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "long"):


## 2 – Modell-Definition

`whisper_flamingo` ist eine in `inference.py` ergänzte Modellklasse
(Klasse `WhisperFlamingoModel`), die neben Audio auch Lip-Crop-Features verarbeitet.
Der Checkpoint liegt lokal unter `model-bin/whisper_flamingo/`.

In [2]:
MODELS = {
    "whisper_flamingo_large": {
        "model_type": "whisper_flamingo",
        "chkpt": "model-bin/whisper_flamingo/whisper-flamingo_en_large_vc2_noisy.pt",
    }
}

## 3 – Sessions & Experimente

Gleiches 5-Session-Subset wie in allen vorherigen Notebooks.
Das Grid (E18–E27) spiegelt exakt das Grid aus `02_` wider,
um Whisper-Flamingo direkt mit BL4 unter gleichen Decoding-Bedingungen zu vergleichen.

In [3]:
SESSION_IDS = ["session_40", "session_43", "session_49", "session_50", "session_54"]

In [4]:
EXPERIMENTS = {
    # 15 Sekunden
    "E18_whisper_flamingo_bs4_len15": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 4,
    "max_length": 15,
    "comment": "Whisper-Flamingo, beam=4, len=15",
    },

    "E19_whisper_flamingo_bs8_len15": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 8,
    "max_length": 15,
    "comment": "Whisper-Flamingo, beam=8, len=15",
    },
    
    "E20_whisper_flamingo_bs12_len15": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 12,
    "max_length": 15,
    "comment": "Whisper-Flamingo, beam=12, len=15",
    },

    # 18 Sekunden
    "E21_whisper_flamingo_bs4_len18": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 4,
    "max_length": 18,
    "comment": "Whisper-Flamingo, beam=4, len=18",
    },

    "E22_whisper_flamingo_bs8_len18": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 8,
    "max_length": 18,
    "comment": "Whisper-Flamingo, beam=8, len=18",
    },
    
    "E23_whisper_flamingo_bs12_len18": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 12,
    "max_length": 18,
    "comment": "Whisper-Flamingo, beam=12, len=18",
    },

    # 20 Sekunden
    "E24_whisper_flamingo_bs4_len20": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 4,
    "max_length": 20,
    "comment": "Whisper-Flamingo, beam=4, len=20",
    },

    "E25_whisper_flamingo_bs8_len20": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 8,
    "max_length": 20,
    "comment": "Whisper-Flamingo, beam=8, len=20",
    },

    "E26_whisper_flamingo_bs12_len20": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 12,
    "max_length": 20,
    "comment": "Whisper-Flamingo, beam=12, len=20",
    },

    "E27_whisper_flamingo_bs8_len10": {
    "base_model": "whisper_flamingo_large",
    "beam_size": 8,
    "max_length": 10,
    "comment": "Whisper-Flamingo, beam=8, len=10",
    },
}


## 4 – Inference

10 Experimente × 5 Sessions = 50 Läufe.

In [5]:
for sid in SESSION_IDS:
    session_dir = f"data-bin/dev/{sid}"
    print(f"\n########## Starte Experimente für {sid} ##########")

    for exp_name in EXPERIMENTS:
        run_inference_for_experiment(
            exp_name=exp_name,
            base_models=MODELS,
            experiments=EXPERIMENTS,
            session_dir=session_dir,
        )


########## Starte Experimente für session_40 ##########

Starte Inference für Experiment: E18_whisper_flamingo_bs4_len15
  base_model      = whisper_flamingo_large
  model_type      = whisper_flamingo
  checkpoint_path = model-bin/whisper_flamingo/whisper-flamingo_en_large_vc2_noisy.pt
  beam_size       = 4
  max_length      = 15
  output_dir_name = output_E18_whisper_flamingo_bs4_len15
  session_dir     = data-bin/dev_without_central_videos/dev/session_40
  comment         = Whisper-Flamingo, beam=4, len=15
Loading whisper_flamingo model...
Loading Whisper-Flamingo AV model (large-v2) on cuda


/home/josch080/Projektgruppe/mcorec_baseline/third_party/whisper-flamingo/whisper/__init__.py:157: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, 

Whisper dropout rate : 0.0
Loading AV-HuBERT encoder


/home/josch080/Projektgruppe/mcorec_baseline/third_party/av_hubert/fairseq/fairseq/checkpoint_utils.py:315: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(

Using AV-HuBERT encoder with parameters: 325136104
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Adding gated x attn layers
Loading Whisper-Flamingo checkpoint weights from model-bin/whisper_flamingo/whisper-f

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 29/29 [00:50<00:00,  1.74s/it]

Processing speakers:  20%|██        | 1/5 [01:09<04:39, 69.99s/it]


Processing speakers:  40%|████      | 2/5 [02:29<03:47, 75.73s/it]


Processing speakers:  60%|██████    | 3/5 [03:34<02:21, 70.60s/it]


Processing speakers:  80%|████████  | 4/5 [05:03<01:18, 78.16s/it]


Processing speakers: 100%|██████████| 5/5 [05:52<00:00, 70.57s/it]


## 5 – Evaluation & Aggregation

In [8]:
# Ergebnisse aller 10 Experimente auswerten und an gemeinsame CSV anhängen
df_dev = append_eval_results_for_experiments(
    experiments=EXPERIMENTS,
    session_ids=SESSION_IDS,
    target_csv="results_dev_subset_by_session.csv",
)



########## Evaluate für session_40 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_wf/bin/python script/evaluate.py --session_dir data-bin/dev_without_central_videos/dev/session_40 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_40 ===

--- Evaluating output dir: output_E01_bs4_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.564, 'spk_1': 0.4281, 'spk_2': 0.5576, 'spk_3': 0.4283, 'spk_4': 0.4793, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.282, 'spk_1': 0.21405, 'spk_2': 0.2788, 'spk_3': 0.21415, 'spk_4': 0.23965, 'spk_5': 0.20945}

--- Evaluating output dir: output_E02_bs8_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.561, 'spk_1': 0.4312, 'spk_2': 0.5506, 'spk_3': 0.4283, 'spk_4': 0.5041, 'spk_5': 0.4189}
Speaker clustering F

## 6 – Ergebnisanalyse

Drei Vergleichsebenen:
1. Whisper-Flamingo intern (alle Konfigurationen)
2. Whisper-Flamingo vs. Whisper Large-v3 (audio-only)
3. Gesamtvergleich aller drei Modell-Familien

In [10]:
import pandas as pd

# Gesamtergebnis einlesen
dev_df = pd.read_csv("results_dev_subset_by_session.csv")

# Whisper-Flamingo intern (E18–E27)
whisper_flamingo_models = [
    "E18_whisper_flamingo_bs4_len15",
    "E19_whisper_flamingo_bs8_len15",
    "E20_whisper_flamingo_bs12_len15",
    "E21_whisper_flamingo_bs4_len18",
    "E22_whisper_flamingo_bs8_len18",
    "E23_whisper_flamingo_bs12_len18",
    "E24_whisper_flamingo_bs4_len20",
    "E25_whisper_flamingo_bs8_len20",
    "E26_whisper_flamingo_bs12_len20",
    "E27_whisper_flamingo_bs8_len10",
]

# Über alle Sessions mitteln und nach Joint Error sortieren (niedrigster = bester)
wf_df = (
    dev_df[dev_df["model"].isin(whisper_flamingo_models)]
    .groupby("model")[["avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .reset_index()
)

print("Whisper-Flamingo (aggregiert über dev-Subset):")
display(wf_df.sort_values("avg_joint_error"))

# Referenz-Modelle aus 02_ und 02c_
whisper_models = ["E16_whisper_bs8_len20", "E17_whisper_bs12_len20"]
best_avsr_models = ["E08_bs8_len20", "E09_bs12_len20"]

baseline_models = whisper_models + best_avsr_models

baseline_df = (
    dev_df[dev_df["model"].isin(baseline_models)]
    .groupby("model")[["avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .reset_index()
)

print("Bisherige Baselines (Whisper + AVSR-Finetune):")
display(baseline_df.sort_values("avg_joint_error"))

# Gesamtvergleich mit Modell-Familie als Label
wf_df2 = wf_df.copy()
wf_df2["family"] = "Whisper-Flamingo"

baseline_df2 = baseline_df.copy()
# Modell-Familie anhand des Präfixes bestimmen
baseline_df2["family"] = baseline_df2["model"].apply(
    lambda m: "Whisper" if m.startswith("E16") or m.startswith("E17") else "AVSR-Finetune"
)

# Alle drei Familien in einer Tabelle zusammenführen
comparison_df = (
    pd.concat([wf_df2, baseline_df2], ignore_index=True)
    .sort_values("avg_joint_error")
    .reset_index(drop=True)
)

print("Gesamtvergleich:")
display(comparison_df)


Whisper-Flamingo (aggregiert über dev-Subset):


,model,avg_speaker_wer,avg_joint_error
1,E19_whisper_flamingo_bs8_len15,0.921862,0.537123
4,E22_whisper_flamingo_bs8_len18,0.923613,0.537999
9,E27_whisper_flamingo_bs8_len10,0.923795,0.538090
7,E25_whisper_flamingo_bs8_len20,0.925969,0.539176
5,E23_whisper_flamingo_bs12_len18,0.931475,0.541930
2,E20_whisper_flamingo_bs12_len15,0.932060,0.542222
0,E18_whisper_flamingo_bs4_len15,0.933546,0.542965
8,E26_whisper_flamingo_bs12_len20,0.933904,0.543144
3,E21_whisper_flamingo_bs4_len18,0.935684,0.544034
6,E24_whisper_flamingo_bs4_len20,0.938264,0.545324


Bisherige Baselines (Whisper + AVSR-Finetune):


,model,avg_speaker_wer,avg_joint_error
1,E09_bs12_len20,0.495416,0.323900
0,E08_bs8_len20,0.495798,0.324091
3,E17_whisper_bs12_len20,1.025096,0.588740
2,E16_whisper_bs8_len20,1.031613,0.591999


Gesamtvergleich:


,model,avg_speaker_wer,avg_joint_error,family
0,E09_bs12_len20,0.495416,0.323900,AVSR-Finetune
1,E08_bs8_len20,0.495798,0.324091,AVSR-Finetune
2,E19_whisper_flamingo_bs8_len15,0.921862,0.537123,Whisper-Flamingo
3,E22_whisper_flamingo_bs8_len18,0.923613,0.537999,Whisper-Flamingo
4,E27_whisper_flamingo_bs8_len10,0.923795,0.538090,Whisper-Flamingo
5,E25_whisper_flamingo_bs8_len20,0.925969,0.539176,Whisper-Flamingo
6,E23_whisper_flamingo_bs12_len18,0.931475,0.541930,Whisper-Flamingo
7,E20_whisper_flamingo_bs12_len15,0.932060,0.542222,Whisper-Flamingo
8,E18_whisper_flamingo_bs4_len15,0.933546,0.542965,Whisper-Flamingo
9,E26_whisper_flamingo_bs12_len20,0.933904,0.543144,Whisper-Flamingo


## 7 – Interpretation

### Whisper-Flamingo intern

Die WER liegt für alle Konfigurationen im engen Bereich 0.92–0.94,
der Hyperparameter-Einfluss ist gering. Beste Konfiguration: `beam=8, len=15` (E19).
Anders als bei BL4 bringt `len=20` hier keine konsistente Verbesserung.

### Vergleich der Modell-Familien

| Modell | WER | Joint Error | rel. Δ vs. BL4 |
|--------|-----|-------------|----------------|
| Whisper Large-v3 (audio-only) | ~1.03 | ~0.59 | +108 % |
| Whisper-Flamingo (beste Konfig.) | ~0.92 | ~0.54 | +86 % |
| BL4 AV-HuBERT (beam=12, len=20) | ~0.495 | ~0.324 | – |

Whisper-Flamingo profitiert von den visuellen Features (~10 % relative WER-Reduktion
gegenüber Whisper), bleibt aber fast doppelt so fehleranfällig wie BL4.

**Warum ist BL4 so viel besser?**
- AV-HuBERT wurde explizit für audio-visuelles Sprecher-Tracking entwickelt
- Das MCoRec-Fine-Tuning passt das Modell auf die spezifische Domäne (überlappende Konversation) an
- Whisper-Flamingo hat kein vergleichbares In-Domain-Training

**Schlussfolgerung:** Whisper-Flamingo wird nicht weiterverfolgt.
BL4 bleibt das Referenzmodell. Als nächster Ansatz wird LLM-basiertes
Postprocessing der BL4-Transkriptionen getestet (`02f_` ff.).
